In [2]:
import os
import torch
from dataloader import MRDataset
from tqdm import tqdm
from tqdm import tqdm_notebook
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
import torch.nn as nn
from torch.utils import data
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import functional as F
import io
import requests
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
import cv2
import pdb
import model

In [16]:
train_dataset = MRDataset(list(range(0,1130)), '/home/niamh/Documents/MRNET/data/', 
                    'acl',
                   'axial',
                        transform=None)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=1, shuffle=False, num_workers=11, drop_last=False)


mod =torch.load('/home/niamh/Documents/MRNET/final_lg/con/acl/axial/secondtime/model_fold4_aa_acl_axial_test_auc_0.9714_train_auc_0.9805_val_auc_0.9532_epoch_14_best_test.pth',
             map_location=torch.device('cpu'))

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

mod._modules.get('pretrained_model').layer4.register_forward_hook(hook_feature);

features_blobs_conv = []
def hook_feature2(module, input, output):
    features_blobs_conv.append(output.data.cpu().numpy())

mod._modules.get('conv').register_forward_hook(hook_feature2);

features_blobs_gr = []

def backward_hook(module, grad_in, grad_out):
    print(grad_out)
    features_blobs_gr.append(grad_out[0].cpu().numpy() )
    
mod._modules.get('pretrained_model').layer4.register_backward_hook(backward_hook);



In [17]:
def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
    if torch.cuda.is_available():
        a = a.cuda()
      #  dim = torch.Tensor(dim).cuda()
        order_index = order_index.cuda()
    return torch.index_select(a, dim, order_index)



In [ ]:

for i, (image,label, weight) in enumerate(train_loader):
      #  optimizer.zero_grad()
    if len(str(i)) == 1:
        string = '000' + str(i) 
    elif len(str(i)) == 2:
        string = '00' + str(i) 
    elif len(str(i)) == 3:
        string = '0' + str(i) 
    elif len(str(i)) == 4:
        string = str(i) 
        
    task = 'acl'
    plane ='axial'
    os.mkdir('./cams/' + task + '/' + plane +'/' + string)
    label = label[0]
    weight = weight[0]
    a = np.zeros((1,1))
    prediction = mod.forward(image.float())
    loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction.squeeze(0), label)
    loss.backward()
    
    weights =nn.AdaptiveAvgPool2d(1)(torch.Tensor(features_blobs_gr[i]))
    
    a = torch.Tensor(features_blobs_conv[i])
    x = torch.Tensor(features_blobs[i])
    a =  nn.Softmax(2)(a.view(*a.size()[:2], -1)).view_as(a)
    m = torch.max(a.flatten(2), 2).values
    b= tile(m, 1, 64)
    c = a.flatten(2).flatten(1) / b
    d = torch.reshape(c, (a.shape[0],512,8,8))
    o = x*d 
    
    
    gcam = torch.mul(o, weights).sum(dim=1, keepdim=True)
    gcam = F.relu(gcam)
    gcam = F.interpolate(
                gcam, (256,256), mode="bilinear", align_corners=False
            )
    B, C, H, W = gcam.shape


    gcam = gcam.view(B, -1)
    gcam -= gcam.min(dim=1, keepdim=True)[0]
    gcam /= gcam.max(dim=1, keepdim=True)[0]
    gcam = gcam.view(B, H, W)

    
    for j in range(0, image.shape[1]):
        img = image[0][j].numpy()
        img = img.transpose(1, 2, 0)
        heatmap = np.uint8(255 * gcam.numpy()[j])

        heatmap = (cv2
                    .cvtColor(cv2.applyColorMap(
                        cv2.resize(heatmap, (256, 256)),
                        cv2.COLORMAP_JET), 
                               cv2.COLOR_BGR2RGB)
                  )
        result = heatmap * 0.5 + img * 0.5  

        pil_img_cam = Image.fromarray(np.uint8(result))
        pil_img_cam.save('./cams/' + task + '/' + plane +'/' + string +'/' +str(j) + '.png')
        





## grad cams for ACL sagittal 

In [ ]:
train_dataset = MRDataset(list(range(0,1130)), '/home/niamh/Documents/MRNET/data/', 
                    'acl',
                   'sagittal',
                        transform=None)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=1, shuffle=False, num_workers=11, drop_last=False)


mod =torch.load('/home/niamh/Documents/MRNET/final_lg/con/acl/sagittal/second_time/model_fold0_naa_acl_sagittal_test_auc_0.9285_val_auc_0.8534_train_auc_0.9820_epoch_17.pth',
             map_location=torch.device('cpu'))

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

mod._modules.get('pretrained_model').layer4.register_forward_hook(hook_feature);

features_blobs_conv = []
def hook_feature2(module, input, output):
    features_blobs_conv.append(output.data.cpu().numpy())

mod._modules.get('conv').register_forward_hook(hook_feature2);

features_blobs_gr = []

def backward_hook(module, grad_in, grad_out):
    print(grad_out)
    features_blobs_gr.append(grad_out[0].cpu().numpy() )
    
mod._modules.get('pretrained_model').layer4.register_backward_hook(backward_hook);



for i, (image,label, weight) in enumerate(train_loader):
    if (i <38) & (i>=21 ):
 
      #  optimizer.zero_grad()
        if len(str(i)) == 1:
            string = '000' + str(i) 
        elif len(str(i)) == 2:
            string = '00' + str(i) 
        elif len(str(i)) == 3:
            string = '0' + str(i) 
        elif len(str(i)) == 4:
            string = str(i) 

        task = 'acl'
        plane ='sagittal'
        try:
            os.mkdir('./cams/' + task + '/' + plane +'/' + string)
        except:
            print('')
            
        label = label[0]
        weight = weight[0]
        a = np.zeros((1,1))
        prediction = mod.forward(image.float())
        loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction.squeeze(0), label)
        loss.backward()

        weights =nn.AdaptiveAvgPool2d(1)(torch.Tensor(features_blobs_gr[i-21]))

        a = torch.Tensor(features_blobs_conv[i-21])
        x = torch.Tensor(features_blobs[i-21])
        a =  nn.Softmax(2)(a.view(*a.size()[:2], -1)).view_as(a)
        m = torch.max(a.flatten(2), 2).values
        b= tile(m, 1, 64)
        c = a.flatten(2).flatten(1) / b
        d = torch.reshape(c, (a.shape[0],512,8,8))
        o = x*d 


        gcam = torch.mul(o, weights).sum(dim=1, keepdim=True)
        gcam = F.relu(gcam)
        gcam = F.interpolate(
                    gcam, (256,256), mode="bilinear", align_corners=False
                )
        B, C, H, W = gcam.shape


        gcam = gcam.view(B, -1)
        gcam -= gcam.min(dim=1, keepdim=True)[0]
        gcam /= gcam.max(dim=1, keepdim=True)[0]
        gcam = gcam.view(B, H, W)


        for j in range(0, image.shape[1]):
            img = image[0][j].numpy()
            img = img.transpose(1, 2, 0)
            heatmap = np.uint8(255 * gcam.numpy()[j])

            heatmap = (cv2
                        .cvtColor(cv2.applyColorMap(
                            cv2.resize(heatmap, (256, 256)),
                            cv2.COLORMAP_JET), 
                                   cv2.COLOR_BGR2RGB)
                      )
            result = heatmap * 0.5 + img * 0.5  

            pil_img_cam = Image.fromarray(np.uint8(result))
            pil_img_cam.save('./cams/' + task + '/' + plane +'/' + string +'/' +str(j) + '.png')
    elif i > 38:
        break






## grad cams for ACL SAGITTAL - not multyplying by attention mask

In [ ]:
train_dataset = MRDataset(list(range(0,1130)), '/home/niamh/Documents/MRNET/data/', 
                    'acl',
                   'sagittal',
                        transform=None)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=1, shuffle=False, num_workers=11, drop_last=False)


mod =torch.load('/home/niamh/Documents/MRNET/final_lg/con/acl/sagittal/second_time/model_fold0_naa_acl_sagittal_test_auc_0.9285_val_auc_0.8534_train_auc_0.9820_epoch_17.pth',
             map_location=torch.device('cpu'))

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

mod._modules.get('pretrained_model').layer4.register_forward_hook(hook_feature);

features_blobs_gr = []

def backward_hook(module, grad_in, grad_out):
    print(grad_out)
    features_blobs_gr.append(grad_out[0].cpu().numpy() )
    
mod._modules.get('pretrained_model').layer4.register_backward_hook(backward_hook);


for i, (image,label, weight) in enumerate(train_loader):
    if (i >38):
        break
      #  optimizer.zero_grad()
    if len(str(i)) == 1:
        string = '000' + str(i) 
    elif len(str(i)) == 2:
        string = '00' + str(i) 
    elif len(str(i)) == 3:
        string = '0' + str(i) 
    elif len(str(i)) == 4:
        string = str(i) 
        
    task = 'acl'
    plane ='sagittal'
    try:
        os.mkdir('./cams/' + task + '/' + plane +'2/' + string)
    except:
        print('')
        
    label = label[0]
    weight = weight[0]
    prediction = mod.forward(image.float())
    loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction.squeeze(0), label)
    loss.backward()
    
    weights =nn.AdaptiveAvgPool2d(1)(torch.Tensor(features_blobs_gr[i]))
    
    gcam = torch.mul(torch.Tensor(features_blobs[i]), weights).sum(dim=1, keepdim=True)
    gcam = F.relu(gcam)
    gcam = F.interpolate(
                gcam, (256,256), mode="bilinear", align_corners=False
            )
    B, C, H, W = gcam.shape


    gcam = gcam.view(B, -1)
    gcam -= gcam.min(dim=1, keepdim=True)[0]
    gcam /= gcam.max(dim=1, keepdim=True)[0]
    gcam = gcam.view(B, H, W)

    
    for j in range(0, image.shape[1]):
        img = image[0][j].numpy()
        img = img.transpose(1, 2, 0)
        heatmap = np.uint8(255 * gcam.numpy()[j])

        heatmap = (cv2
                    .cvtColor(cv2.applyColorMap(
                        cv2.resize(heatmap, (256, 256)),
                        cv2.COLORMAP_JET), 
                               cv2.COLOR_BGR2RGB)
                  )
        result = heatmap * 0.5 + img * 0.5  

        pil_img_cam = Image.fromarray(np.uint8(result))
        pil_img_cam.save('./cams/' + task + '/' + plane +'2/' + string +'/' +str(j) + '.png')
        

